# Creating SQLite database from csv files

Reference:

https://discussions.udacity.com/t/creating-db-file-from-csv-files-with-non-ascii-unicode-characters/174958/7

#### Create database and all tables

Import modules needed:

In [1]:
import sqlite3
import csv
from pprint import pprint

Connect to the database (if it doesn't exist, it will be created in the folder that your notebook is in):

In [2]:
sqlite_file = "osm_atx.db"

In [3]:
conn = sqlite3.connect(sqlite_file)

Create the cursor object

In [4]:
cur = conn.cursor()

Create the table from nodes.csv:

In [6]:
cur.execute('''CREATE TABLE nodes(id INTEGER PRIMARY KEY, lat REAL, lon REAL, user TEXT, uid INTEGER, version TEXT, 
                changeset INTEGER, timestamp TEXT)''')
cur.execute('''CREATE TABLE IN NOT EXISTS nodes_tags(id INTEGER PRIMARY KEY, key TEXT, value TEXT, type TEXT, FOREIGN KEY (id) REFERENCES nodes (id))''')

ERROR: An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (2, 135))



OperationalError: table nodes already exists

In [7]:
cur.execute('''CREATE TABLE nodes_tags(id INTEGER PRIMARY KEY, key TEXT, value TEXT, type TEXT, FOREIGN KEY (id) REFERENCES nodes (id))''')

In [8]:
cur.execute('''CREATE TABLE ways(id INTEGER PRIMARY KEY, user TEXT, uid INTEGER, version TEXT, changeset INTEGER, timestamp TEXT, FOREIGN KEY(id) REFERENCES nodes (id),
                FOREIGN KEY (user) REFERENCES nodes (user), FOREIGN KEY (uid) REFERENCES nodes (uid))''')

In [10]:
cur.execute('''CREATE TABLE ways_tags(id INTEGER PRIMARY KEY, key TEXT, value TEXT, type TEXT, FOREIGN KEY (id) REFERENCES ways (id))''')

In [12]:
cur.execute('''CREATE TABLE ways_nodes(id INTEGER PRIMARY KEY, node_id INTEGER, position INTEGER, FOREIGN KEY (id) REFERENCES ways(id), FOREIGN KEY (id) REFERENCES ways_tags (id))''')

Commit the changes

In [13]:
conn.commit()

#### Read in csv file as a dictionary, format the data as a list of tuples, for nodes table:


In [14]:
with open('nodes.csv', 'rb') as fin:
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db = [(i['id'], i['lat'], i['lon'], i['user'].decode('utf-8'), i['uid'], i['version'].decode('utf-8'),
              i['changeset'], i['timestamp'].decode('utf-8')) for i in dr]
    # ".decode('utf-8')" is used mostly for text datatype otherwise, a ProgrammingError will 
    # sometimes occur during insertion of data
    # https://discussions.udacity.com/t/case-study-quiz-preparing-for-database-help/169697/19

Insert data

In [15]:
cur.executemany("INSERT INTO nodes(id, lat, lon, user, uid, version, changeset, timestamp) VALUES (?, ?, ?, ?, ?, ?, ?, ?);", to_db)

Commit the changes

In [16]:
conn.commit()

Checking that the data imported correctly. (Since I don't want to make my notebook too large, I will just check this in sqlite, in the command line.)

In [17]:
cur.execute('SELECT * FROM nodes limit 20')

In [18]:
first20 = cur.fetchall()

In [19]:
print 'first 20:'
pprint(first20)

first 20:
[(26546004,
  30.4695355,
  -97.7972587,
  u'Tylan',
  388279,
  u'15',
  8497118,
  u'2011-06-20T18:36:15Z'),
 (26546005,
  30.4713386,
  -97.7975919,
  u'APD',
  105002,
  u'19',
  767484,
  u'2009-03-09T09:03:33Z'),
 (26546006,
  30.4711721,
  -97.798579,
  u'HJD',
  75480,
  u'24',
  533807,
  u'2009-02-18T16:44:40Z'),
 (26546008,
  30.469115,
  -97.7966751,
  u'claysmalley',
  119881,
  u'28',
  13420621,
  u'2012-10-09T01:08:42Z'),
 (26546009,
  30.4688175,
  -97.7976688,
  u'Tylan',
  388279,
  u'38',
  8497118,
  u'2011-06-20T18:22:32Z'),
 (26546010,
  30.469413,
  -97.797558,
  u'Tylan',
  388279,
  u'17',
  8497118,
  u'2011-06-20T18:36:15Z'),
 (26546011,
  30.4714758,
  -97.7980443,
  u'Tylan',
  388279,
  u'4',
  8497118,
  u'2011-06-20T18:36:15Z'),
 (26546012,
  30.4714208,
  -97.798367,
  u'Tylan',
  388279,
  u'14',
  8497118,
  u'2011-06-20T18:36:15Z'),
 (26546025,
  30.4751578,
  -97.799145,
  u'richlv',
  47892,
  u'24',
  18948024,
  u'2013-11-17T08:34:54Z'

#### Read in nodes_tags table data, inserting data

In [20]:
with open('nodes_tags.csv', 'rb') as f:
    d = csv.DictReader(f) 
    tdb = [(i['id'], i['key'].decode('utf-8'), i['value'].decode('utf-8'), i['type'].decode('utf-8')) for i in d]

In [21]:
cur.executemany("INSERT INTO nodes_tags(id, key, value, type) VALUES (?, ?, ?, ?);", tdb)

IntegrityError: UNIQUE constraint failed: nodes_tags.id

In [22]:
tdb

[('26546008', u'highway', u'traffic_signals', u'regular'),
 ('26546009', u'highway', u'traffic_signals', u'regular'),
 ('26546041', u'highway', u'traffic_signals', u'regular'),
 ('26546043', u'highway', u'traffic_signals', u'regular'),
 ('26546065', u'highway', u'traffic_signals', u'regular'),
 ('26546067', u'highway', u'traffic_signals', u'regular'),
 ('26546082', u'noref', u'yes', u'regular'),
 ('26546082', u'exit_to', u'Avery Ranch Boulevard', u'regular'),
 ('26546082', u'highway', u'motorway_junction', u'regular'),
 ('26546091', u'noref', u'yes', u'regular'),
 ('26546091', u'exit_to', u'Avery Ranch Boulevard', u'regular'),
 ('26546091', u'highway', u'motorway_junction', u'regular'),
 ('26546111', u'noref', u'yes', u'regular'),
 ('26546111', u'highway', u'motorway_junction', u'regular'),
 ('26546144', u'noref', u'yes', u'regular'),
 ('26546144', u'exit_to', u'Brushy Creek Road;Cypress Creek Road', u'regular'),
 ('26546144', u'highway', u'motorway_junction', u'regular'),
 ('26546151'

In [23]:
conn.commit(())

TypeError: commit() takes no arguments (1 given)

checked on sqlite command line if there is data in the nodes_tags table, but didn't see anything. So I tried to execute the command below:

In [24]:
conn.commit()

After this, I checked the contents of nodes_tags in sqlite command line again and saw some data in it. 

I did the DELETE FROM table; to remove the data from nodes_tags table in sqlite command line.

Checked contents and it's empty.

I need to check tdb:

In [25]:
tdb

[('26546008', u'highway', u'traffic_signals', u'regular'),
 ('26546009', u'highway', u'traffic_signals', u'regular'),
 ('26546041', u'highway', u'traffic_signals', u'regular'),
 ('26546043', u'highway', u'traffic_signals', u'regular'),
 ('26546065', u'highway', u'traffic_signals', u'regular'),
 ('26546067', u'highway', u'traffic_signals', u'regular'),
 ('26546082', u'noref', u'yes', u'regular'),
 ('26546082', u'exit_to', u'Avery Ranch Boulevard', u'regular'),
 ('26546082', u'highway', u'motorway_junction', u'regular'),
 ('26546091', u'noref', u'yes', u'regular'),
 ('26546091', u'exit_to', u'Avery Ranch Boulevard', u'regular'),
 ('26546091', u'highway', u'motorway_junction', u'regular'),
 ('26546111', u'noref', u'yes', u'regular'),
 ('26546111', u'highway', u'motorway_junction', u'regular'),
 ('26546144', u'noref', u'yes', u'regular'),
 ('26546144', u'exit_to', u'Brushy Creek Road;Cypress Creek Road', u'regular'),
 ('26546144', u'highway', u'motorway_junction', u'regular'),
 ('26546151'

In [26]:
cur.executemany("INSERT INTO nodes_tags(id, key, value, type) VALUES (?, ?, ?, ?);", tdb)

IntegrityError: UNIQUE constraint failed: nodes_tags.id

In [27]:
conn.commit()

In [28]:
cur.execute("DROP TABLE nodes_tags")

In [29]:
conn.commit()

In [30]:
cur.execute('''CREATE TABLE nodes_tags(id INTEGER, key TEXT, value TEXT, type TEXT, FOREIGN KEY (id) REFERENCES nodes (id))''')

In [32]:
cur.executemany("INSERT INTO nodes_tags(id, key, value, type) VALUES (?, ?, ?, ?);", tdb)

In [33]:
conn.commit()

In [34]:
cur.execute("SELECT * FROM nodes_tags LIMIT 10")

In [35]:
print cur.fetchall()

[(26546008, u'highway', u'traffic_signals', u'regular'), (26546009, u'highway', u'traffic_signals', u'regular'), (26546041, u'highway', u'traffic_signals', u'regular'), (26546043, u'highway', u'traffic_signals', u'regular'), (26546065, u'highway', u'traffic_signals', u'regular'), (26546067, u'highway', u'traffic_signals', u'regular'), (26546082, u'noref', u'yes', u'regular'), (26546082, u'exit_to', u'Avery Ranch Boulevard', u'regular'), (26546082, u'highway', u'motorway_junction', u'regular'), (26546091, u'noref', u'yes', u'regular')]


Now that nodes_tags table is good, we start to insert the ways data:

In [36]:
with open('ways.csv', 'rb') as g:
    r = csv.DictReader(g) 
    to_ways = [(i['id'], i['user'].decode('utf-8'), i['uid'], i['version'].decode('utf-8'), i['changeset'], i['timestamp'].decode('utf-8')) for i in r]

In [37]:
cur.executemany("INSERT INTO ways(id, user, uid, version, changeset, timestamp) VALUES (?, ?, ?, ?, ?, ?);", to_ways)

In [38]:
conn.commit()

In [39]:
cur.execute("SELECT * FROM ways LIMIT 10")

In [40]:
cur.fetchall()

[(4358672, u'claysmalley', 119881, u'19', 10974544, u'2012-03-14T05:28:59Z'),
 (4358673, u'NE2', 207745, u'8', 10343099, u'2012-01-09T16:14:46Z'),
 (4358675, u'claysmalley', 119881, u'8', 10212599, u'2011-12-27T02:55:41Z'),
 (4358677, u'tompkinsjs', 517514, u'12', 15084279, u'2013-02-19T01:43:36Z'),
 (4531212, u'claysmalley', 119881, u'10', 10974544, u'2012-03-14T05:28:59Z'),
 (4531214, u'ChrisZontine', 1376118, u'6', 25101150, u'2014-08-29T16:39:04Z'),
 (4531215, u'ChrisZontine', 1376118, u'5', 16556381, u'2013-06-14T21:27:39Z'),
 (4531217, u'Cam4rd98', 1132286, u'14', 15537280, u'2013-03-29T12:05:08Z'),
 (4531218, u'nammala', 3479270, u'13', 40336781, u'2016-06-28T07:21:55Z'),
 (4531219, u'Iowa Kid', 703517, u'8', 17120434, u'2013-07-28T03:13:58Z')]

After making sure that ways table is good, we now take care of ways_tags:

In [41]:
with open('ways_tags.csv', 'rb') as h:
    s = csv.DictReader(h)
    to_ways_tags = [(i['id'], i['key'].decode('utf-8'), i['value'].decode('utf-8'), i['type'].decode('utf-8')) for i in s]

In [42]:
cur.execute("INSERT INTO ways_tags(id, key, value, type) VALUES (?, ?, ?, ?);", to_ways_tags)

ProgrammingError: Incorrect number of bindings supplied. The current statement uses 4, and there are 2049840 supplied.

In [43]:
conn.commit()

In [44]:
cur.execute("DROP TABLE ways_tags")

In [45]:
cur.execute('''CREATE TABLE ways_tags(id INTEGER, key TEXT, value TEXT, type TEXT, FOREIGN KEY (id) REFERENCES ways (id))''')

In [49]:
cur.executemany("INSERT INTO ways_tags(id, key, value, type) VALUES (?, ?, ?, ?);", to_ways_tags)

In [50]:
conn.commit()

In [51]:
cur.execute("SELECT * from ways_tags limit 10")

In [52]:
cur.fetchall()

[(4358672, u'NHS', u'yes', u'regular'),
 (4358672, u'ref', u'US 183', u'regular'),
 (4358672, u'toll', u'no', u'regular'),
 (4358672, u'lanes', u'3', u'regular'),
 (4358672, u'layer', u'1', u'regular'),
 (4358672, u'bridge', u'yes', u'regular'),
 (4358672, u'oneway', u'yes', u'regular'),
 (4358672, u'highway', u'motorway', u'regular'),
 (4358672, u'old_ref', u'SH 29', u'regular'),
 (4358673, u'NHS', u'yes', u'regular')]

Lastly, the ways_nodes table:

In [57]:
with open("ways_nodes.csv", "rb") as j:
    t = csv.DictReader(j)
    to_ways_nodes = [(i['id'], i['node_id'], i['position']) for i in t]

In [58]:
cur.executemany("INSERT INTO ways_nodes(id, node_id, position) VALUES (?, ?, ?);", to_ways_nodes)

IntegrityError: UNIQUE constraint failed: ways_nodes.id

In [59]:
conn.commit()

In [60]:
cur.execute("DROP TABLE ways_nodes")

In [61]:
conn.commit()

In [62]:
cur.execute("""CREATE TABLE ways_nodes(id INTEGER, node_id INTEGER, position INTEGER, FOREIGN KEY (id) REFERENCES ways (id))""")

In [63]:
cur.executemany("INSERT INTO ways_nodes(id, node_id, position) VALUES (?, ?, ?);", to_ways_nodes)

In [64]:
conn.commit()

In [65]:
cur.execute("SELECT * FROM ways_nodes limit 10")

In [66]:
cur.fetchall()

[(4358672, 26546039, 0),
 (4358672, 1674200199, 1),
 (4358673, 26546203, 0),
 (4358673, 1332583222, 1),
 (4358675, 1332583222, 0),
 (4358675, 26546004, 1),
 (4358677, 26546038, 0),
 (4358677, 26546046, 1),
 (4358677, 26546048, 2),
 (4531212, 1619623774, 0)]

In [67]:
conn.close()